<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-18 10:28:09
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.37 C
-------------------
Today PnL: 2.59 L (1.94%)
Current PnL: -18.19 L (-12.38%)
CY Booked + Current PnL: -9.92 L (-6.75%)
-------------------
Total profit:  3.63 L
Total loss:  -21.82 L
-------------------
Total Booked + Current PnL: 17.06 L (14.21%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.05%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 77.67 L (56.82%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.32%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-17.42,H-LC,6.54,113896.0,9584.0,7062.0,2.39,9.19,6.20,15.96,11.0,1.36,0.85,27.64,X5K,ATH,METALS
4,APOLLOHOSP,7098.95,8285.00,-12.98,H-LC,6.29,173822.0,17645.0,8448.0,1.02,11.30,4.86,16.71,19.0,2.09,1.30,30.54,X40N,BTT,HEALTHCARE
68,SBIN,760.30,863.00,-11.34,M-LC,3.87,219685.0,18206.0,9007.0,0.30,9.04,4.10,13.51,61.0,2.02,1.64,22.77,XY25,NTT,BANKS
51,LALPATHLAB,2838.53,3545.00,-22.63,H-SC,10.88,231023.0,38003.0,10026.0,1.48,19.69,4.34,24.89,134.0,3.79,1.72,47.63,X40N,NTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,10.21,H-MC,13.90,157412.0,31212.0,14970.0,-1.24,24.73,9.51,36.59,111.0,2.08,1.17,40.35,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJOOENG,102.50,143.33,-32.41,H-SC,6.04,102320.0,-180.0,41010.0,-0.23,-0.18,40.08,39.83,116.0,-0.00,0.76,1.31,AR,ATH,MISC
58,PGHH,13388.00,18062.58,-30.55,H-MC,6.25,201165.0,345.0,69764.0,1.08,0.17,34.68,34.92,82.0,0.00,1.50,4.86,X40,ATH,FMCG
56,NATIONALUM,189.63,247.44,-49.46,H-MC,1.99,100191.0,-313.0,30949.0,0.78,-0.31,30.89,30.49,80.0,-0.01,0.75,31.99,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-19.78,H-LC,2.80,213939.0,9439.0,31984.0,-0.15,4.62,14.95,20.25,10.0,0.30,1.59,13.75,X40N,BTT,PHARMA
57,NESTLEIND,1134.30,1377.0,-11.79,H-LC,3.44,271651.0,6225.0,50554.0,6.56,2.35,18.61,21.40,8.0,0.12,2.02,9.85,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-10.32,H-LC,3.70,175025.0,-26361.0,113976.0,1.79,-13.09,65.12,43.51,27.0,-0.23,1.30,21.84,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-7.23,H-LC,3.73,183942.0,-16368.0,76354.0,0.03,-8.17,41.51,29.94,17.0,-0.21,1.37,27.08,X200,ATH,IT
37,HINDUNILVR,2413.81,2723.0,-8.70,H-LC,5.93,287280.0,16933.0,17696.0,3.40,6.26,6.16,12.81,28.0,0.96,2.14,20.15,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELAXO,902.64,1176.00,-34.94,H-SC,10.66,60358.0,-55180.0,90169.0,8.00,-47.76,149.39,30.28,136.0,-0.61,0.45,17.92,X40N,NTT,FOOTWEAR
15,BLUESTARCO,1646.70,2326.38,10.86,H-MC,6.57,191380.0,26710.0,41262.0,7.20,16.22,21.56,41.28,91.0,0.65,1.43,25.50,X40N,ATH,AC
13,BATAINDIA,1550.24,2096.00,-37.58,M-SC,7.45,93325.0,-35345.0,80642.0,7.12,-27.47,86.41,35.20,209.0,-0.44,0.70,7.12,X40,NTT,FOOTWEAR
35,HEROMOTOCO,4311.81,5949.07,-4.90,H-MC,8.64,176400.0,25487.0,31823.0,7.05,16.89,18.04,37.97,92.0,0.80,1.31,46.47,AR,ATH,AUTO
57,NESTLEIND,1134.30,1377.00,-11.79,H-LC,3.44,271651.0,6225.0,50554.0,6.56,2.35,18.61,21.40,8.0,0.12,2.02,9.85,XY25,NTT,FMCG


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,EASEMYTRIP,18.12,26.40,-8.83,M-SC,25.40,87920.0,-88895.0,169694.0,-1.96,-50.28,193.01,45.70,197.0,-0.52,0.66,2.27,XY24,NTT,TRAVEL
55,MEDANTA,1087.93,1486.00,10.21,H-MC,13.90,157412.0,31212.0,14970.0,-1.24,24.73,9.51,36.59,111.0,2.08,1.17,40.35,XY24,NTT,HEALTHCARE
88,VALIANTORG,512.64,838.00,-323.33,H-SC,16.40,94900.0,-38386.0,122981.0,-0.96,-28.80,129.59,63.47,143.0,-0.31,0.71,59.08,XR,NTT,CHEMICALS
73,SONACOMS,578.05,1006.00,-36.95,M-SC,13.59,78435.0,-22724.0,97612.0,-0.81,-22.46,124.45,74.03,203.0,-0.23,0.58,10.60,AR,BTT,AUTO
89,VBL,492.64,671.64,-11.61,H-LC,10.05,268650.0,3610.0,92684.0,-0.50,1.36,34.50,36.33,5.0,0.04,2.00,14.84,X40N,ATH,FMCG


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,95.30,M-SC,10.14,84181.0,-16596.0,16693.0,1.73,-16.47,19.83,0.09,243.0,-0.99,0.63,10.59,OX40N,NTT,DURABLES
20,CERA,7989.07,9475.0,-18.38,H-SC,3.13,116577.0,-27226.0,53975.0,1.23,-18.93,46.30,18.60,148.0,-0.50,0.87,28.65,OX40N,NTT,CERAMICS
84,UJJIVANSFB,52.77,60.0,80.86,H-SC,20.12,114372.0,-28107.0,47625.0,1.66,-19.73,41.64,13.70,163.0,-0.59,0.85,34.99,OX40N,NTT,BANKS
43,INDIGOPNTS,1407.73,1408.0,112.61,M-SC,2.58,139525.0,-35034.0,35063.0,1.63,-20.07,25.13,0.02,221.0,-1.00,1.04,20.99,OX40N,NTT,PAINTS
49,KANSAINER,299.63,340.0,-68.89,H-SC,3.02,214281.0,-55386.0,91712.0,1.12,-20.54,42.80,13.47,142.0,-0.60,1.60,8.38,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.69,H-LC,16.09,235622.0,-60312.0,154215.0,-0.05,-20.38,65.45,31.73,2.0,-0.39,1.76,0.59,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.03,H-LC,11.25,258190.0,-3411.0,149027.0,-0.37,-1.30,57.72,55.67,4.0,-0.02,1.92,4.72,X40,BTT,IT
89,VBL,492.64,671.64,-11.61,H-LC,10.05,268650.0,3610.0,92684.0,-0.50,1.36,34.50,36.33,5.0,0.04,2.00,14.84,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.01,H-LC,10.80,196950.0,-19569.0,112478.0,0.40,-9.04,57.11,42.91,7.0,-0.17,1.47,3.14,AR,NTT,ELECTRICAL
57,NESTLEIND,1134.30,1377.00,-11.79,H-LC,3.44,271651.0,6225.0,50554.0,6.56,2.35,18.61,21.40,8.0,0.12,2.02,9.85,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,10.21,H-MC,13.90,157412.0,31212.0,14970.0,-1.24,24.73,9.51,36.59,111.0,2.08,1.17,40.35,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-11.03,H-LC,6.66,222235.0,37754.0,32624.0,4.07,20.46,14.68,38.14,35.0,1.16,1.66,33.24,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.35,11.32,H-SC,11.73,132489.0,7192.0,51816.0,3.36,5.74,39.11,47.09,141.0,0.14,0.99,65.82,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,10.86,H-MC,6.57,191380.0,26710.0,41262.0,7.20,16.22,21.56,41.28,91.0,0.65,1.43,25.50,X40N,ATH,AC
55,MEDANTA,1087.93,1486.00,10.21,H-MC,13.90,157412.0,31212.0,14970.0,-1.24,24.73,9.51,36.59,111.0,2.08,1.17,40.35,XY24,NTT,HEALTHCARE
90,VOLTAS,1278.28,1918.49,-3.32,H-MC,7.67,202725.0,10983.0,85043.0,6.02,5.73,41.95,50.08,101.0,0.13,1.51,12.91,XY25,ATH,AC
19,CAMS,3643.00,5211.76,-4.28,H-SC,4.21,108864.0,6860.0,37068.0,0.50,6.73,34.05,43.06,123.0,0.19,0.81,24.99,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-4.90,H-MC,8.64,176400.0,25487.0,31823.0,7.05,16.89,18.04,37.97,92.0,0.80,1.31,46.47,AR,ATH,AUTO
50,KPIGREEN,497.21,731.35,11.32,H-SC,11.73,132489.0,7192.0,51816.0,3.36,5.74,39.11,47.09,141.0,0.14,0.99,65.82,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-51.84,H-SC,11.79,134360.0,11024.0,122738.0,-0.42,8.94,91.35,108.45,119.0,0.09,1.00,33.29,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-42.43,M-SC,4.53,97290.0,5792.0,72958.0,4.55,6.33,74.99,86.07,219.0,0.08,0.73,40.74,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-12.89,M-LC,7.59,153419.0,2474.0,106979.0,0.26,1.64,69.73,72.51,54.0,0.02,1.14,7.76,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.42,73920.0,-39629.0,79678.0,1.42,-34.90,107.79,35.27,268.0,-0.50,0.55,80.92,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-2.49,H-SC,15.41,90816.0,-10147.0,104675.0,1.87,-10.05,115.26,93.63,149.0,-0.10,0.68,35.27,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-17.80,H-SC,13.27,93300.0,-4680.0,26553.0,3.49,-4.78,28.46,22.32,152.0,-0.18,0.70,35.32,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-323.33,H-SC,16.40,94900.0,-38386.0,122981.0,-0.96,-28.80,129.59,63.47,143.0,-0.31,0.71,59.08,XR,NTT,CHEMICALS
9,BAJAJFINSV,1994.80,2758.00,-8.22,H-LC,7.23,98343.0,598.0,36800.0,4.25,0.61,37.42,38.26,38.0,0.02,0.73,29.46,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-4.90,H-MC,8.64,176400.0,25487.0,31823.0,7.05,16.89,18.04,37.97,92.0,0.80,1.31,46.47,AR,ATH,AUTO
51,LALPATHLAB,2838.53,3545.00,-22.63,H-SC,10.88,231023.0,38003.0,10026.0,1.48,19.69,4.34,24.89,134.0,3.79,1.72,47.63,X40N,NTT,HEALTHCARE
28,DMART,3764.92,5201.00,-11.03,H-LC,6.66,222235.0,37754.0,32624.0,4.07,20.46,14.68,38.14,35.0,1.16,1.66,33.24,X40N,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.00,-12.98,H-LC,6.29,173822.0,17645.0,8448.0,1.02,11.30,4.86,16.71,19.0,2.09,1.30,30.54,X40N,BTT,HEALTHCARE
5,ASIANPAINT,2961.56,4250.00,-10.32,H-LC,3.70,175025.0,-26361.0,113976.0,1.79,-13.09,65.12,43.51,27.0,-0.23,1.30,21.84,X40,BTT,PAINTS


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.57
1,25,41.34
2,50,71.21


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.92
LC    34.77
MC    26.31
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.48
X40N     15.53
X40      13.24
XY25     11.67
XR        8.95
AR        8.29
OX40N     7.38
X5K       2.24
MH        1.74
X200      1.37
SR        1.11
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.54
H-SC    24.21
H-MC    23.40
M-SC    13.25
M-LC     7.17
M-MC     2.57
L-SC     1.46
L-LC     1.06
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.15,-1.39,33.98
IT,11.72,-18.63,77.88
FINANCE,8.18,-18.73,62.66
BANKS,7.51,-13.27,61.37
MISC,5.76,-18.77,86.86
PAINTS,5.57,-16.70,39.69
HEALTHCARE,5.26,6.91,16.59
ELECTRICAL,4.69,-11.66,60.01
AUTO,4.52,-11.32,59.76


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2814105.0,23
XR,1096019.0,13
AR,1095990.0,10
X40,795388.0,10
X40N,523932.0,12
XY25,481375.0,8
OX40N,468546.0,10
SR,194725.0,2
X5K,137660.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2769479.0,28
M-SC,1470469.0,17
H-MC,1238721.0,18
H-LC,1174106.0,18
M-LC,415879.0,5
M-MC,332094.0,2
L-SC,264804.0,3
L-MC,57421.0,1
L-LC,43886.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      927653.0      6
M-SC       XY24      858778.0      7
H-SC       AR        671042.0      5
           XR        519109.0      6
H-LC       X40       510182.0      5
H-MC       XY24      486431.0      5
M-MC       XY24      332094.0      2
H-LC       AR        232791.0      2
           X40N      208360.0      5
H-MC       XY25      206210.0      2
H-SC       OX40N     205452.0      4
H-MC       X40       204564.0      4
H-SC       X40N      199682.0      4
           SR        194725.0      2
L-SC       XR        175031.0      2
M-LC       XY24      169295.0      2
M-SC       AR        160334.0      2
M-LC       X5K       130598.0      1
M-SC       OX40N     126035.0      4
           XY25      122769.0      1
           XR        121911.0      2
H-MC       X40N      115890.0      3
           XR        115568.0      1
M-LC       XR        106979.0      1
H-LC       XY25       99503.0      3
L-SC       OX40N      89773.0      1
M-SC       X40        80642.0      1
H-LC       X200       76354.0      1
L-MC       XR         57421.0      1
H-SC       MH         51816.0      1
H-MC       OX40N      47286.0      1
L-LC       XY25       43886.0      1
H-LC       XY24       39854.0      1
H-MC       AR         31823.0      1
           MH         30949.0      1
M-LC       XY25        9007.0      1
H-LC       X5K         7062.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
